# Code Europe 2025 - File Search Demo
This notebook shows how to build an **Azure AI Foundry Agent** that can search across uploaded files. This sample is based on one of the official Microsoft Learn samples - https://aka.ms/IntroToAgentService/Sample

### Prerequisites
```bash
pip install azure-ai-projects azure-ai-agents azure-identity python-dotenv
```
Set these environment variables (e.g. in a `.env` file):
- `PROJECT_ENDPOINT`
- `MODEL_DEPLOYMENT_NAME`

This example uses Entra ID authentication, so before running, please authenticate:

`az login`

## Import packages and environment variables

In [25]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import FileSearchTool, FilePurpose, ListSortOrder
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv()

True

## Connect to your Azure AI Foundry Project

In [26]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

agents_client = project_client.agents

## Upload a file & create a searchable vector store

In [27]:
file = agents_client.files.upload_and_poll(
    file_path="employee_handbook.pdf",
    purpose=FilePurpose.AGENTS,
)
print(f"Uploaded file - id: {file.id}")

# Build the vector store
vector_store = agents_client.vector_stores.create_and_poll(
    file_ids=[file.id],
    name="my_vectorstore",
)
print(f"Created vector store - id: {vector_store.id}")


Uploaded file - id: assistant-XvMm27s66doUD3isqkKKyh
Created vector store - id: vs_mzxe1qTTzWLmFCsdcVAjMNvk


## Create a file‑search tool and agent

In [28]:
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="DEMO - Code Europe 2025 Agent - SDK",
    instructions=("You are helpful agent and can search information from uploaded files"),
    tools=file_search.definitions,
    tool_resources=file_search.resources,
)
print(f"Created agent - id: {agent.id}")

Created agent - id: asst_AMTUmrXifTw5K5OuuK3ImCR6


## Create a conversation thread, and add a message to it

In [29]:
thread = agents_client.threads.create()
print(f"Created thread - id: {thread.id}")

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are the company values?",
)
print(f"Created message - id: {message.id}")


Created thread - id: thread_QjvKPQM6GACS8Pyg28hB5jCx
Created message - id: msg_Rc43l4Iof5YYwrtlBRtELacI


## Run the agent

In [30]:
run = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_choice="auto"
)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


## Display the conversation history

In [31]:
messages = agents_client.messages.list(
    thread_id=thread.id,
    order=ListSortOrder.ASCENDING,
)

for msg in messages:
    if msg.text_messages:
        last = msg.text_messages[-1]
        print(f"{msg.role}: {last.text.value}")

MessageRole.USER: What are the company values?
MessageRole.AGENT: The company values are:

- Integrity: We demonstrate integrity by adhering to the highest ethical standards and doing the right thing, even when no one is watching.
- Results Oriented: We focus on achieving results and exceeding expectations through effective planning, execution, and accountability.
- Adaptability: We embrace change and continuously evolve to meet the dynamic needs of our clients and the marketplace.
- Client Focused: We build lasting relationships by understanding and anticipating our clients’ needs, providing exceptional service, and consistently delivering on our promises.
- Teamwork: We value collaboration and leverage our collective strengths to achieve shared goals. We support each other and celebrate our successes together.
- Continuous Improvement: We foster a culture of learning, innovation, and improvement. We seek opportunities to grow personally and professionally.

These values form the foun

## Clean‑up resources

In [32]:
agents_client.vector_stores.delete(vector_store.id)
agents_client.files.delete(file_id=file.id)
agents_client.delete_agent(agent.id)
project_client.close()

print("Clean-up completed")

Clean-up completed
